In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,905 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:13 https://developer.download.nv

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-08 01:22:53--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.48MB/s    in 0.2s    

2022-08-08 01:22:54 (4.48 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# READ IN THE REVIEWS DATASET AS A DATAFRANE
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...| 2015-08-31|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...|         Jewelry|          5|    

In [5]:
# MODULE 16 CHALLENGE: DELIVERABLE II
# RE-CREATE THE vine_table DATAFRAME
vine_table = df.select(["review_id", "helpful_votes", "star_rating", "total_votes", "vine", "verified_purchase"])
vine_table.show()

+--------------+-------------+-----------+-----------+----+-----------------+
|     review_id|helpful_votes|star_rating|total_votes|vine|verified_purchase|
+--------------+-------------+-----------+-----------+----+-----------------+
|R135Q3VZ4DQN5N|            0|          5|          0|   N|                Y|
|R2N0QQ6R4T7YRY|            0|          5|          0|   N|                N|
|R3N5JE5Y4T6W5M|            0|          5|          0|   N|                Y|
|R2I150CX5IVY9Q|            0|          5|          0|   N|                Y|
|R1RM9ICOOA9MQ3|            0|          5|          0|   N|                Y|
|R2J2KMDL10UMSH|            0|          5|          0|   N|                Y|
|R3R9ZUFA4TB4FQ|            0|          5|          0|   N|                Y|
|R3UQ8VAQN7R6WL|            0|          5|          0|   N|                Y|
|R1FXZ69C01JNQM|            0|          5|          0|   N|                Y|
| RY36LB4OW0FFS|            0|          5|          0|   N|     

In [6]:
# FILTER THE NEW DATAFRAME
vine_table_filtered = vine_table.filter("total_votes >= 20")
vine_table_filtered.show()

+--------------+-------------+-----------+-----------+----+-----------------+
|     review_id|helpful_votes|star_rating|total_votes|vine|verified_purchase|
+--------------+-------------+-----------+-----------+----+-----------------+
|R16YC6SMGKA8SR|           23|          5|         23|   N|                Y|
| R5O4WE9UM60B0|           29|          1|         29|   N|                Y|
|R2EKRVCRC7U0IY|           22|          1|         24|   N|                Y|
|R2OKV47GETH0L7|           24|          5|         24|   N|                Y|
| R36LII9IITE17|           22|          1|         24|   N|                Y|
|R35VKNE16PFY0H|           22|          5|         24|   N|                N|
|R3TG008LHHZZIN|          123|          5|        125|   N|                Y|
|R2FYQBKCC1XG4M|           20|          5|         20|   N|                Y|
| R63PV336NI5X7|           27|          1|         28|   N|                Y|
|R1R42WPHB5ZSWI|           51|          5|         52|   N|     

In [7]:
# CREATE A NEW FILTERED DATAFRAME TO RETRIEVE helpful_votes WHERE RATIO IS EQUAL TO OR GREATER THAN 50%
helpful_votes_df = vine_table_filtered.filter((vine_table_filtered["helpful_votes"] / vine_table_filtered["total_votes"]) >= .5)
helpful_votes_df.show()

+--------------+-------------+-----------+-----------+----+-----------------+
|     review_id|helpful_votes|star_rating|total_votes|vine|verified_purchase|
+--------------+-------------+-----------+-----------+----+-----------------+
|R16YC6SMGKA8SR|           23|          5|         23|   N|                Y|
| R5O4WE9UM60B0|           29|          1|         29|   N|                Y|
|R2EKRVCRC7U0IY|           22|          1|         24|   N|                Y|
|R2OKV47GETH0L7|           24|          5|         24|   N|                Y|
| R36LII9IITE17|           22|          1|         24|   N|                Y|
|R35VKNE16PFY0H|           22|          5|         24|   N|                N|
|R3TG008LHHZZIN|          123|          5|        125|   N|                Y|
|R2FYQBKCC1XG4M|           20|          5|         20|   N|                Y|
| R63PV336NI5X7|           27|          1|         28|   N|                Y|
|R1R42WPHB5ZSWI|           51|          5|         52|   N|     

In [8]:
# FILTER AND CREATE A DATAFRAME WHERE THERE IS A VINE REVIEW
vine_reviews_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "Y")
vine_reviews_df.show()

+--------------+-------------+-----------+-----------+----+-----------------+
|     review_id|helpful_votes|star_rating|total_votes|vine|verified_purchase|
+--------------+-------------+-----------+-----------+----+-----------------+
|R15PXA9XU9ZKSU|           29|          4|         29|   Y|                N|
|R2LJ4QYQ4PYDEH|           24|          5|         26|   Y|                N|
|R2PJWWQZ37WCIC|           95|          5|         96|   Y|                N|
|R33G2VDKRN2USY|           21|          5|         21|   Y|                N|
|R13R4X37T2U50A|           52|          4|         57|   Y|                N|
|R1X1J2NAQ39DF6|           29|          4|         34|   Y|                N|
|R2UV5XHFNMPXOX|           60|          5|         61|   Y|                N|
|R31V08BCC9X3MF|           18|          5|         22|   Y|                N|
|R2G00QWTYW9N2G|           86|          5|         91|   Y|                N|
| RL8H3NKKP989J|           30|          5|         34|   Y|     

In [9]:
# FILTER AND CREATE A DATAFRAME WHERE THERE IS NOT A VINE REVIEW
non_vine_reviews_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "N")
non_vine_reviews_df.show()

+--------------+-------------+-----------+-----------+----+-----------------+
|     review_id|helpful_votes|star_rating|total_votes|vine|verified_purchase|
+--------------+-------------+-----------+-----------+----+-----------------+
|R16YC6SMGKA8SR|           23|          5|         23|   N|                Y|
| R5O4WE9UM60B0|           29|          1|         29|   N|                Y|
|R2EKRVCRC7U0IY|           22|          1|         24|   N|                Y|
|R2OKV47GETH0L7|           24|          5|         24|   N|                Y|
| R36LII9IITE17|           22|          1|         24|   N|                Y|
|R35VKNE16PFY0H|           22|          5|         24|   N|                N|
|R3TG008LHHZZIN|          123|          5|        125|   N|                Y|
|R2FYQBKCC1XG4M|           20|          5|         20|   N|                Y|
| R63PV336NI5X7|           27|          1|         28|   N|                Y|
|R1R42WPHB5ZSWI|           51|          5|         52|   N|     

In [10]:
# DETERMINE TOTAL NUMBER OF REVIEWS FROM VINE MEMBERS
total_member_reviews = vine_reviews_df.count()
total_member_reviews

21

In [11]:
# DETERMINE TOTAL NUMBER OF 5-STAR REVIEWS FROM VINE MEMBERS
total_member_5Star_reviews = vine_reviews_df.filter(vine_reviews_df.star_rating == 5).count()
total_member_5Star_reviews

11

In [12]:
# DETERMINE PERCENTAGE OF 5-STAR REVIEWS FROM VINE MEMBERS
vine_5Star_percentage = (total_member_5Star_reviews / total_member_reviews) * 100
vine_5Star_percentage

52.38095238095239

In [13]:
# DETERMINE TOTAL NUMBER OF REVIEWS FROM NON-VINE MEMBERS
total_NonMember_reviews = non_vine_reviews_df.count()
total_NonMember_reviews

7689

In [14]:
# DETERMINE TOTAL NUMBER OF 5-STAR REVIEWS FROM NON-VINE MEMBERS
total_NonMember_5Star_reviews = non_vine_reviews_df.filter(non_vine_reviews_df.star_rating == 5).count()
total_NonMember_5Star_reviews

4444

In [15]:
# DETERMINE PERCENTAGE OF 5-STAR REVIEWS FROM NON-VINE MEMBERS
vine_5Star_NonMember_percentage = (total_NonMember_5Star_reviews / total_NonMember_reviews) * 100
vine_5Star_NonMember_percentage

57.79685264663805

In [16]:
# RETRIEVE AND CREATE A DATAFRAME OF VINE PROGRAM ENTRIES WITHOUT A REVIEW
NonVine_reviews_with_purchase = helpful_votes_df.filter((helpful_votes_df["vine"] == "N") & (helpful_votes_df["verified_purchase"] == "Y"))
NonVine_reviews_with_purchase.show()

+--------------+-------------+-----------+-----------+----+-----------------+
|     review_id|helpful_votes|star_rating|total_votes|vine|verified_purchase|
+--------------+-------------+-----------+-----------+----+-----------------+
|R16YC6SMGKA8SR|           23|          5|         23|   N|                Y|
| R5O4WE9UM60B0|           29|          1|         29|   N|                Y|
|R2EKRVCRC7U0IY|           22|          1|         24|   N|                Y|
|R2OKV47GETH0L7|           24|          5|         24|   N|                Y|
| R36LII9IITE17|           22|          1|         24|   N|                Y|
|R3TG008LHHZZIN|          123|          5|        125|   N|                Y|
|R2FYQBKCC1XG4M|           20|          5|         20|   N|                Y|
| R63PV336NI5X7|           27|          1|         28|   N|                Y|
|R1R42WPHB5ZSWI|           51|          5|         52|   N|                Y|
| RXSS0QZJE1TEO|           35|          1|         35|   N|     

In [17]:
# DETERMINE TOTAL NUMBER OF NON-VINE MEMBERS REVIEWS THAT MADE A PURCHASE
NonVine_withPurchase_total_reviews = NonVine_reviews_with_purchase.count()
NonVine_withPurchase_total_reviews

6160

In [18]:
# DETERMINE TOTAL NUMBER OF NON-VINE MEMBER 5-STAR REVIEWS THAT MADE A PURCHASE
NonVine_withPurchase_5Star_totalReviews = NonVine_reviews_with_purchase.filter(non_vine_reviews_df["star_rating"] == 5).count()
NonVine_withPurchase_5Star_totalReviews

3665

In [19]:
# DETERMINE PERCENTAGE OF NON-MEMBER 5-STAR REVIEWS THAT MADE A PURCHASE
NonVine_withPurchase_5Star_Percentage = (NonVine_withPurchase_5Star_totalReviews / NonVine_withPurchase_total_reviews) * 100
NonVine_withPurchase_5Star_Percentage

59.496753246753244